
# Nivel 3 – Memoria en agentes (Notebook end‑to‑end)

Este notebook implementa, paso a paso, técnicas de **memoria de corto y largo plazo** para agentes conversacionales, con ejemplos de:
- **LangChain** para memoria de conversación (buffer y resumen).
- **FAISS** y **ChromaDB** como *vector stores*.
- **Sentence-Transformers** para embeddings semánticos.
- **Conversational Retrieval Chain (RAG)** con LangChain.
- **Persistencia de metadatos** en **SQLite**.
- Utilidades de **evaluación** (recall@k, latencia) y **MMR**.

> Está alineado con el documento “Nivel 3 – Memoria en agentes”. Puedes ejecutar cada sección de forma independiente.



## 0. Requisitos e instalación

Ejecuta esta celda si aún no tienes las dependencias:


In [ ]:

# ¡Ejecuta según necesites! Descomenta para instalar.
# %pip install -q langchain langchain-openai sentence-transformers faiss-cpu chromadb sqlite-utils tiktoken
# Opcional GPU: %pip install -q faiss-gpu
# Opcional (benchmarking): %pip install -q rank-bm25



## 1. Configuración básica

Define claves y modelos. Si usarás `ChatOpenAI`, exporta la variable `OPENAI_API_KEY` o asigna aquí.
Si no tienes API key, verás una ruta alternativa con un **LLM simulado** para poder probar flujos.


In [ ]:

import os

# === Configuración general ===
EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"  # 384-d, rápido y sólido para POC
PERSIST_DIR = "./chroma_mem"

# === OpenAI (opcional para memoria por resumen / RAG) ===
# os.environ["OPENAI_API_KEY"] = "TU_API_KEY_AQUI"  # <- opción 1
# O expórtala en tu entorno antes de abrir el notebook:
# %env OPENAI_API_KEY=TU_API_KEY_AQUI

OPENAI_READY = bool(os.environ.get("OPENAI_API_KEY"))
print("OPENAI_READY:", OPENAI_READY)



## 2. Memoria de corto plazo (LangChain)

Incluye **ConversationBufferWindowMemory** (últimos *k* turnos) y **ConversationSummaryMemory** (resumen).
Si no dispones de LLM de pago, usaremos un **LLM simulado** que devuelve eco/plantillas para demostrar el flujo.


In [ ]:

from langchain.memory import ConversationBufferWindowMemory, ConversationSummaryMemory
from langchain.schema import HumanMessage, AIMessage

# Fallback LLM simulado (si no hay OpenAI). Solo para ilustrar el flujo.
class DummyLLM:
    def __call__(self, *args, **kwargs):
        class R:
            content = "Resumen (dummy): " + str(kwargs.get("input", ""))[:120]
        return R()

if OPENAI_READY:
    from langchain_openai import ChatOpenAI
    llm = ChatOpenAI(temperature=0)
else:
    llm = DummyLLM()

buffer_memory = ConversationBufferWindowMemory(k=4, return_messages=True)
summary_memory = ConversationSummaryMemory(llm=llm)  # requiere un LLM (real o dummy)

# Simulación de diálogo
buffer_memory.save_context(
    {"human": "Quiero vuelos a Madrid"}, 
    {"ai": "¿Desde qué ciudad y fechas?"}
)
buffer_memory.save_context(
    {"human": "Desde CDMX del 12 al 18 de octubre"}, 
    {"ai": "¿Tienes preferencia de aerolínea?"}
)

print("=== Buffer (últimos turnos) ===")
for msg in buffer_memory.load_memory_variables({})["history"]:
    role = "HUMANO" if isinstance(msg, HumanMessage) else "AGENTE"
    print(f"[{role}] {msg.content}")

summary_memory.save_context({"input": "Prefiero pasillo y Aeroméxico"}, {"output": "Entendido, buscando..."})
print("\n=== Resumen (con LLM) ===")
print(summary_memory.load_memory_variables({})["history"])



## 3. Memoria de largo plazo con **FAISS**

Indexamos “episodios” (hechos/decisiones) con **Sentence-Transformers** y consultamos por similitud (coseno normalizado).


In [ ]:

from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

modelo = SentenceTransformer(EMBEDDING_MODEL)

episodios = [
    {"id": 1, "text": "El usuario prefiere asiento de pasillo y equipaje documentado", "user_id": "u123", "ts": 1714800000},
    {"id": 2, "text": "Reservamos hotel en Barcelona el 5 de mayo", "user_id": "u123", "ts": 1714900000},
    {"id": 3, "text": "Tiene tarjetas Visa y prefiere pagar en MXN", "user_id": "u123", "ts": 1715000000},
]

X = modelo.encode([e["text"] for e in episodios], normalize_embeddings=True).astype("float32")
index = faiss.IndexFlatIP(X.shape[1])  # inner product == cosine si normalizado
index.add(X)

meta = {i: episodios[i] for i in range(len(episodios))}

consulta = "¿Qué prefiere sobre asientos el usuario?"
qv = modelo.encode([consulta], normalize_embeddings=True).astype("float32")
scores, ids = index.search(qv, k=3)

print("=== Resultados FAISS ===")
for rank, (i, s) in enumerate(zip(ids[0], scores[0]), start=1):
    if i == -1: 
        continue
    print(f"#{rank} score={s:.3f} →", meta[int(i)]["text"])



### 3.1 MMR (Maximal Marginal Relevance)

Selecciona resultados relevantes **y diversos** para evitar duplicados semánticos.


In [ ]:

def mmr(query_vec: np.ndarray, cand_vecs: np.ndarray, lamb: float = 0.5, k: int = 3):
    # query_vec: (d,), cand_vecs: (n, d) normalizados
    selected = []
    remaining = list(range(cand_vecs.shape[0]))
    sims = cand_vecs @ query_vec  # (n,)
    while remaining and len(selected) < k:
        if not selected:
            # primer pick: mayor similitud con la consulta
            i = int(np.argmax(sims[remaining]))
            selected.append(remaining.pop(i))
        else:
            best, best_score = None, -1e9
            for idx_j, ridx in enumerate(remaining):
                diversity = 0.0
                if selected:
                    diversity = np.max(cand_vecs[selected] @ cand_vecs[ridx])
                score = lamb * sims[ridx] - (1 - lamb) * diversity
                if score > best_score:
                    best_score, best = score, idx_j
            selected.append(remaining.pop(best))
    return selected

# Demostración con los mismos episodios
selected_idxs = mmr(qv[0], X, lamb=0.6, k=2)
print("MMR seleccionó índices:", selected_idxs)
for i in selected_idxs:
    print("→", meta[i]["text"])



## 4. Conversational RAG con **ChromaDB** + **LangChain**

- Ingesta con **RecursiveCharacterTextSplitter** (chunking 600/80).
- Embeddings con **sentence-transformers** (normalizados).
- Retriever con **MMR**.
- Cadena conversacional con **ConversationBufferMemory**.

> Esta sección usa LLM; si no tienes API key, el flujo de RAG se montará pero la llamada al modelo real no se ejecutará. 


In [ ]:

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.memory import ConversationBufferMemory

# Datos de ejemplo (políticas y preferencias)
docs = [
    ("POLÍTICAS", "Los cambios de vuelo permiten una modificación sin costo en tarifa flexible. Reembolsos sujetos a reglas."),
    ("PREFERENCIAS", "El usuario prefiere asientos de pasillo, equipaje documentado y pagar en MXN."),
]

# 1) Chunking
splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=80)
chunks = []
for title, text in docs:
    for i, ch in enumerate(splitter.split_text(text)):
        chunks.append({"page_content": ch, "metadata": {"title": title, "i": i}})

# 2) Embeddings + Chroma (persistente)
emb = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL, encode_kwargs={"normalize_embeddings": True})
vs = Chroma.from_documents(documents=[type("Doc", (), c)() for c in chunks], embedding=emb, persist_directory=PERSIST_DIR)

# 3) Retriever MMR
retriever = vs.as_retriever(search_type="mmr", search_kwargs={"k": 4, "fetch_k": 12})

# 4) Cadena conversacional (si hay LLM)
if OPENAI_READY:
    from langchain_openai import ChatOpenAI
    from langchain.chains import ConversationalRetrievalChain

    llm = ChatOpenAI(temperature=0)
    conv_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    qa = ConversationalRetrievalChain.from_llm(llm, retriever, memory=conv_memory)

    pregunta = "¿Puedo cambiar de vuelo sin pagar?"
    respuesta = qa({"question": pregunta})
    print("PREGUNTA:", pregunta)
    print("RESPUESTA:", respuesta["answer"])
else:
    print("OPENAI_READY=False → Saltando la llamada al LLM real. El retriever está inicializado y listo.")
    # Demostración: recuperar documentos sin generar respuesta
    demo = retriever.get_relevant_documents("¿Puedo cambiar de vuelo sin pagar?")
    print("Documentos recuperados (títulos):", [d.metadata.get("title") for d in demo])



## 5. Persistencia de **metadatos** en **SQLite**

Guarda episodios y referencia cruzada hacia tu vector store. Mantén **TTL**, permisos y PII fuera del índice vectorial.


In [ ]:

import sqlite3, time

conn = sqlite3.connect("memoria.db")
cur = conn.cursor()
cur.execute("""
CREATE TABLE IF NOT EXISTS episodios (
  id INTEGER PRIMARY KEY,
  user_id TEXT,
  ts INTEGER,
  texto TEXT,
  etiqueta TEXT,
  vector_id TEXT
);
""")

fila = ("u123", int(time.time()), "Reservamos hotel en Barcelona el 5 de mayo", "booking", "faiss:2")
cur.execute("INSERT INTO episodios(user_id, ts, texto, etiqueta, vector_id) VALUES (?,?,?,?,?)", fila)
conn.commit()

# Consulta rápida
for r in cur.execute("SELECT id, user_id, ts, etiqueta, vector_id FROM episodios ORDER BY id DESC LIMIT 3"):
    print(r)
conn.close()



## 6. Evaluación (recall@k, latencia) y utilidades


In [ ]:

import time
import numpy as np

def recall_at_k(relevantes_ids, recuperados_ids, k=3):
    rec = set(recuperados_ids[:k])
    rel = set(relevantes_ids)
    if not rel: 
        return 0.0
    return len(rec & rel) / len(rel)

# Simulación de relevancia (para el ejemplo de FAISS)
relevantes = [0]  # asumimos que el índice 0 del FAISS era relevante
recuperados = list(ids[0])  # de la consulta FAISS previa
print("recall@1:", recall_at_k(relevantes, recuperados, k=1))
print("recall@3:", recall_at_k(relevantes, recuperados, k=3))

# Latencia sencilla
t0 = time.time()
_ = index.search(qv, k=5)
dur_ms = (time.time() - t0) * 1000
print(f"Latencia FAISS (ms): {dur_ms:.2f}")



## 7. Plantilla de **política de memoria** (TTL, PII, seguridad)

- **Scope**: qué datos se guardan (preferencias, decisiones, IDs de reserva), qué se **excluye** (PII sensible sin consentimiento).
- **TTL** por tipo de dato: p. ej., 30 días para episodios; 365 días para preferencias explícitas.
- **Consentimiento**: registrar consentimiento y permitir **borrado** (“derecho al olvido”).
- **Seguridad**: cifrado en reposo y en tránsito; control de acceso por `user_id/org_id`.
- **Trazabilidad**: quién y cuándo leyó/escribió en memoria (logs de auditoría).
- **Re-indexado**: proceso cuando cambia el modelo de embeddings (versionado de índices).



## 8. (Opcional) Recuperación híbrida

Combina recuperación **lexical** (BM25) con **vectorial**. Útil para códigos, números de reserva y OOV.
Si instalas `rank-bm25`, puedes fusionar rankings con estrategias como **Reciprocal Rank Fusion (RRF)**.


In [ ]:

# Ejemplo ilustrativo (requiere: %pip install rank-bm25)
# from rank_bm25 import BM25Okapi
# corpus_tokens = [doc.split() for doc in [e["text"] for e in episodios]]
# bm25 = BM25Okapi(corpus_tokens)
# query = "preferencia asiento usuario"
# scores_bm25 = bm25.get_scores(query.split())
# # Combinar con similitud de FAISS (ya calculada) usando RRF/u otra técnica.
# print("BM25 scores:", scores_bm25)



---
*Generado automáticamente el 2025-08-18 01:11:42.*  
Sugerencias: añade tus propios datasets, activa filtros por metadatos y mide **recall@k** vs. experiencia de usuario.
